# Environment Creation

本文档概述了创建新环境以及 PettingZoo 中为创建新环境而设计的相关有用包装器、实用程序和测试。

我们将逐步创建一个简单的剪刀石头布环境，并提供AEC和并行环境的示例代码。代理环境循环（“AEC”）

请参阅我们的自定义环境教程，了解创建自定义环境的完整演练，包括复杂的环境逻辑和非法操作屏蔽。

## Example Custom Environment

这是一个经过仔细注释的 PettingZoo 石头剪刀布环境版本。

In [10]:
import functools

import gymnasium
import numpy as np
from gymnasium.spaces import Discrete

from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector, wrappers

ROCK = 0
PAPER = 1
SCISSORS = 2
NONE = 3
MOVES = ["ROCK", "PAPER", "SCISSORS", "None"]
NUM_ITERS = 100
REWARD_MAP = {
    (ROCK, ROCK): (0, 0),
    (ROCK, PAPER): (-1, 1),
    (ROCK, SCISSORS): (1, -1),
    (PAPER, ROCK): (1, -1),
    (PAPER, PAPER): (0, 0),
    (PAPER, SCISSORS): (-1, 1),
    (SCISSORS, ROCK): (-1, 1),
    (SCISSORS, PAPER): (1, -1),
    (SCISSORS, SCISSORS): (0, 0),
}


def rps_env(render_mode=None):
    """
    The env function often wraps the environment in wrappers by default.
    You can find full documentation for these methods
    elsewhere in the developer documentation.
    """
    internal_render_mode = render_mode if render_mode != "ansi" else "human"
    env = raw_env(render_mode=internal_render_mode)
    # This wrapper is only for environments which print results to the terminal
    if render_mode == "ansi":
        env = wrappers.CaptureStdoutWrapper(env)
    # this wrapper helps error handling for discrete action spaces
    env = wrappers.AssertOutOfBoundsWrapper(env)
    # Provides a wide vareity of helpful user errors
    # Strongly recommended
    env = wrappers.OrderEnforcingWrapper(env)
    return env


class raw_env(AECEnv):
    """
    The metadata holds environment constants. From gymnasium, we inherit the "render_modes",
    metadata which specifies which modes can be put into the render() method.
    At least human mode should be supported.
    The "name" metadata allows the environment to be pretty printed.
    """

    metadata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and
         should define the following attributes:
        - possible_agents
        - render_mode

        Note: as of v1.18.1, the action_spaces and observation_spaces attributes are deprecated.
        Spaces should be defined in the action_space() and observation_space() methods.
        If these methods are not overridden, spaces will be inferred from self.observation_spaces/action_spaces, raising a warning.

        These attributes should not be changed after initialization.
        """
        self.possible_agents = ["player_" + str(r) for r in range(2)]

        # optional: a mapping between agent name and ID
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )

        # optional: we can define the observation and action spaces here as attributes to be used in their corresponding methods
        self._action_spaces = {agent: Discrete(3) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Discrete(4) for agent in self.possible_agents
        }
        self.render_mode = render_mode

    # Observation space should be defined here.
    # lru_cache allows observation and action spaces to be memoized, reducing clock cycles required to get each agent's space.
    # If your spaces change over time, remove this line (disable caching).
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        return Discrete(4)

    # Action space should be defined here.
    # If your spaces change over time, remove this line (disable caching).
    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return Discrete(3)

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        if self.render_mode is None:
            gymnasium.logger.warn(
                "You are calling render method without specifying any render mode."
            )
            return

        if len(self.agents) == 2:
            string = "Current state: Agent1: {} , Agent2: {}".format(
                MOVES[self.state[self.agents[0]]], MOVES[self.state[self.agents[1]]]
            )
        else:
            string = "Game over"
        print(string)

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """
        # observation of one agent is the previous state of the other
        return np.array(self.observations[agent])

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: NONE for agent in self.agents}
        self.observations = {agent: NONE for agent in self.agents}
        self.num_moves = 0
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            # handles stepping an agent which is already dead
            # accepts a None action for the one agent, and moves the agent_selection to
            # the next dead agent,  or if there are no more dead agents, to the next live agent
            self._was_dead_step(action)
            return

        agent = self.agent_selection

        # the agent which stepped last had its _cumulative_rewards accounted for
        # (because it was returned by last()), so the _cumulative_rewards for this
        # agent should start again at 0
        self._cumulative_rewards[agent] = 0

        # stores action of current agent
        self.state[self.agent_selection] = action

        # collect reward if it is the last agent to act
        if self._agent_selector.is_last():
            # rewards for all agents are placed in the .rewards dictionary
            self.rewards[self.agents[0]], self.rewards[self.agents[1]] = REWARD_MAP[
                (self.state[self.agents[0]], self.state[self.agents[1]])
            ]

            self.num_moves += 1
            # The truncations dictionary must be updated for all players.
            self.truncations = {
                agent: self.num_moves >= NUM_ITERS for agent in self.agents
            }

            # observe the current state
            for i in self.agents:
                self.observations[i] = self.state[
                    self.agents[1 - self.agent_name_mapping[i]]
                ]
        else:
            # necessary so that observe() returns a reasonable observation at all times.
            self.state[self.agents[1 - self.agent_name_mapping[agent]]] = NONE
            # no rewards are allocated until both players give an action
            self._clear_rewards()

        # selects the next agent.
        self.agent_selection = self._agent_selector.next()
        # Adds .rewards to ._cumulative_rewards
        self._accumulate_rewards()

        if self.render_mode == "human":
            self.render()

要与自定义 AEC 环境交互，请使用以下代码：

In [12]:
env = rps_env(render_mode="human")
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # this is where you would insert your policy
        action = env.action_space(agent).sample()

    env.step(action)
env.close()

Current state: Agent1: PAPER , Agent2: None
Current state: Agent1: PAPER , Agent2: PAPER
Current state: Agent1: ROCK , Agent2: None
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: ROCK , Agent2: None
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: SCISSORS , Agent2: None
Current state: Agent1: SCISSORS , Agent2: SCISSORS
Current state: Agent1: PAPER , Agent2: None
Current state: Agent1: PAPER , Agent2: PAPER
Current state: Agent1: SCISSORS , Agent2: None
Current state: Agent1: SCISSORS , Agent2: PAPER
Current state: Agent1: SCISSORS , Agent2: None
Current state: Agent1: SCISSORS , Agent2: ROCK
Current state: Agent1: ROCK , Agent2: None
Current state: Agent1: ROCK , Agent2: PAPER
Current state: Agent1: ROCK , Agent2: None
Current state: Agent1: ROCK , Agent2: ROCK
Current state: Agent1: ROCK , Agent2: None
Current state: Agent1: ROCK , Agent2: ROCK
Current state: Agent1: ROCK , Agent2: None
Current state: Agent1: ROCK , Agent2: ROCK
Current st

## Example Custom Parallel Environment

In [ ]:
import functools

import gymnasium
from gymnasium.spaces import Discrete

from pettingzoo import ParallelEnv
from pettingzoo.utils import parallel_to_aec, wrappers

ROCK = 0
PAPER = 1
SCISSORS = 2
NONE = 3
MOVES = ["ROCK", "PAPER", "SCISSORS", "None"]
NUM_ITERS = 100
REWARD_MAP = {
    (ROCK, ROCK): (0, 0),
    (ROCK, PAPER): (-1, 1),
    (ROCK, SCISSORS): (1, -1),
    (PAPER, ROCK): (1, -1),
    (PAPER, PAPER): (0, 0),
    (PAPER, SCISSORS): (-1, 1),
    (SCISSORS, ROCK): (-1, 1),
    (SCISSORS, PAPER): (1, -1),
    (SCISSORS, SCISSORS): (0, 0),
}


def env(render_mode=None):
    """
    The env function often wraps the environment in wrappers by default.
    You can find full documentation for these methods
    elsewhere in the developer documentation.
    """
    internal_render_mode = render_mode if render_mode != "ansi" else "human"
    env = raw_env(render_mode=internal_render_mode)
    # This wrapper is only for environments which print results to the terminal
    if render_mode == "ansi":
        env = wrappers.CaptureStdoutWrapper(env)
    # this wrapper helps error handling for discrete action spaces
    env = wrappers.AssertOutOfBoundsWrapper(env)
    # Provides a wide vareity of helpful user errors
    # Strongly recommended
    env = wrappers.OrderEnforcingWrapper(env)
    return env


def raw_env(render_mode=None):
    """
    To support the AEC API, the raw_env() function just uses the from_parallel
    function to convert from a ParallelEnv to an AEC env
    """
    env = parallel_env(render_mode=render_mode)
    env = parallel_to_aec(env)
    return env


class parallel_env(ParallelEnv):
    metadata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and should define the following attributes:
        - possible_agents
        - render_mode

        Note: as of v1.18.1, the action_spaces and observation_spaces attributes are deprecated.
        Spaces should be defined in the action_space() and observation_space() methods.
        If these methods are not overridden, spaces will be inferred from self.observation_spaces/action_spaces, raising a warning.

        These attributes should not be changed after initialization.
        """
        self.possible_agents = ["player_" + str(r) for r in range(2)]

        # optional: a mapping between agent name and ID
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )
        self.render_mode = render_mode

    # Observation space should be defined here.
    # lru_cache allows observation and action spaces to be memoized, reducing clock cycles required to get each agent's space.
    # If your spaces change over time, remove this line (disable caching).
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        return Discrete(4)

    # Action space should be defined here.
    # If your spaces change over time, remove this line (disable caching).
    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return Discrete(3)

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        if self.render_mode is None:
            gymnasium.logger.warn(
                "You are calling render method without specifying any render mode."
            )
            return

        if len(self.agents) == 2:
            string = "Current state: Agent1: {} , Agent2: {}".format(
                MOVES[self.state[self.agents[0]]], MOVES[self.state[self.agents[1]]]
            )
        else:
            string = "Game over"
        print(string)

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the `agents` attribute and must set up the
        environment so that render(), and step() can be called without issues.
        Here it initializes the `num_moves` variable which counts the number of
        hands that are played.
        Returns the observations for each agent
        """
        self.agents = self.possible_agents[:]
        self.num_moves = 0
        observations = {agent: NONE for agent in self.agents}
        infos = {agent: {} for agent in self.agents}
        self.state = observations

        return observations, infos

    def step(self, actions):
        """
        step(action) takes in an action for each agent and should return the
        - observations
        - rewards
        - terminations
        - truncations
        - infos
        dicts where each dict looks like {agent_1: item_1, agent_2: item_2}
        """
        # If a user passes in actions with no agents, then just return empty observations, etc.
        if not actions:
            self.agents = []
            return {}, {}, {}, {}, {}

        # rewards for all agents are placed in the rewards dictionary to be returned
        rewards = {}
        rewards[self.agents[0]], rewards[self.agents[1]] = REWARD_MAP[
            (actions[self.agents[0]], actions[self.agents[1]])
        ]

        terminations = {agent: False for agent in self.agents}

        self.num_moves += 1
        env_truncation = self.num_moves >= NUM_ITERS
        truncations = {agent: env_truncation for agent in self.agents}

        # current observation is just the other player's most recent action
        observations = {
            self.agents[i]: int(actions[self.agents[1 - i]])
            for i in range(len(self.agents))
        }
        self.state = observations

        # typically there won't be any information in the infos, but there must
        # still be an entry for each agent
        infos = {agent: {} for agent in self.agents}

        if env_truncation:
            self.agents = []

        if self.render_mode == "human":
            self.render()
        return observations, rewards, terminations, truncations, infos

要与自定义并行环境交互，请使用以下代码：

In [14]:
env = parallel_env(render_mode="human")
observations, infos = env.reset()

while env.agents:
    # this is where you would insert your policy
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}

    observations, rewards, terminations, truncations, infos = env.step(actions)
env.close()

Current state: Agent1: PAPER , Agent2: SCISSORS
Current state: Agent1: PAPER , Agent2: ROCK
Current state: Agent1: SCISSORS , Agent2: PAPER
Current state: Agent1: ROCK , Agent2: PAPER
Current state: Agent1: SCISSORS , Agent2: ROCK
Current state: Agent1: SCISSORS , Agent2: SCISSORS
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: SCISSORS , Agent2: SCISSORS
Current state: Agent1: PAPER , Agent2: ROCK
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: SCISSORS , Agent2: SCISSORS
Current state: Agent1: ROCK , Agent2: SCISSORS
Current state: Agent1: PAPER , Agent2: SCISSORS
Current state: Agent1: SCISSORS , Agent2: PAPER
Current state: Agent1: ROCK , Agent2: PAPER
Current state: Agent1: PAPER , Agent2: ROCK
Current state: Agent1: ROCK , Agent2: ROCK
Current state: Agent1: PAPER , Agent2: PAPER
Current state: Agent1: PAPER , Agent2: PAPER
Current state: Agent1: ROCK , Agent2: PAPER
Current state: Agent1: 

# Using Wrappers
包装器是一种环境转换，它将环境作为输入，并输出与输入环境类似的新环境，但应用了一些转换或验证。 PettingZoo 提供了用于在 AEC API 和并行 API 之间来回转换环境的包装器，以及一组提供输入验证和其他方便的可重用逻辑的简单实用程序包装器。 PettingZoo 还通过 SuperSuit 配套包 ( pip install supersuit ) 包含包装器。

In [ ]:
from pettingzoo.butterfly import pistonball_v6
from pettingzoo.utils import ClipOutOfBoundsWrapper

env = pistonball_v6.env()
wrapped_env = ClipOutOfBoundsWrapper(env)
# Wrapped environments must be reset before use
wrapped_env.reset()

# Developer Utils
utils 目录包含一些有助于调试环境的函数。这些都记录在 API 文档中。
utils 目录还包含一些仅对开发新环境有帮助的类。这些记录如下。

## Agent selector

agent_selector类循环遍历代理


In [ ]:
from pettingzoo.utils import agent_selector
agents = ["agent_1", "agent_2", "agent_3"]
selector = agent_selector(agents)
agent_selection = selector.reset()
# agent_selection will be "agent_1"
for i in range(100):
    agent_selection = selector.next()
    # will select "agent_2", "agent_3", "agent_1", "agent_2", "agent_3", ..."

## Testing Environments

PettingZoo 通过了多项环境合规性测试。如果您要添加新环境，我们鼓励您在自己的环境中运行这些测试。


## API Test
PettingZoo 的 API 有许多功能和要求。为了确保您的环境与 API 一致，我们有 api_test。下面是一个例子：

In [ ]:
from pettingzoo.test import api_test
from pettingzoo.butterfly import pistonball_v6
env = pistonball_v6.env()
api_test(env, num_cycles=1000, verbose_progress=False)

In [ ]:
您只需将环境传递给测试即可。该测试将asssert或给出有关 API 问题的其他错误，如果通过则将正常返回。

可选参数是：

num_cycles ：运行环境这么多周期并检查输出是否与 API 一致。

verbose_progress ：打印出消息以指示测试部分完成。对于调试环境很有用。

## 并行 API 测试
适用于并行环境。

In [ ]:
from pettingzoo.test import parallel_api_test
from pettingzoo.butterfly import pistonball_v6
env = pistonball_v6.parallel_env()
parallel_api_test(env, num_cycles=1000)

## Seed Test
为了拥有一个利用随机性的正确可重现环境，您需要能够通过为定义随机行为的随机数生成器设置种子来使其在评估过程中具有确定性。种子测试检查使用常量调用seed()方法实际上使环境具有确定性。
种子测试采用创建 pettingzoo 环境的函数。例如

In [ ]:
from pettingzoo.test import seed_test, parallel_seed_test
from pettingzoo.butterfly import pistonball_v6
env_fn = pistonball_v6.env
seed_test(env_fn, num_cycles=10)

# or for parallel environments
parallel_env_fn = pistonball_v6.parallel_env
parallel_seed_test(parallel_env_fn)

在内部，有两个单独的测试。
环境播种后，两个单独的环境是否会给出相同的结果？
在调用seed()然后调用reset()之后，单个环境是否会给出相同的结果？

第一个可选参数num_cycles指示环境将运行多长时间来检查确定性。有些环境仅在初始化后很长时间才通过测试。

第二个可选参数test_kept_state允许用户禁用第二个测试。一些基于物理的环境未能通过此测试，因为缓存等造成的差异几乎无法检测到，而这些差异并不重要。


## Max Cycles Test 最大循环测试
最大周期测试测试max_cycles环境参数是否存在以及生成的环境实际运行了正确的周期数。

如果您的环境不采用max_cycles参数，则不应运行此测试。该测试存在的原因是在实现max_cycles时可能会出现许多相差一的错误。测试用法示例如下：

from pettingzoo.test import max_cycles_test
from pettingzoo.butterfly import pistonball_v6
max_cycles_test(pistonball_v6)


## Render Test

渲染测试检查渲染

 1) 不会崩溃，2) 在给定模式时生成正确类型的输出（仅支持'human' 、 'ansi'和'rgb_array'模式）。


In [ ]:
from pettingzoo.test import render_test
from pettingzoo.butterfly import pistonball_v6
env_func = pistonball_v6.env
render_test(env_func)

渲染测试方法采用可选参数custom_tests ，该参数允许在非标准模式下进行附加测试。

custom_tests = {
    "svg": lambda render_result: return isinstance(render_result, str)
}
render_test(env, custom_tests=custom_tests)

## Performance Benchmark Test 性能基准测试

为了确保我们不会出现性能下降，我们进行了性能基准测试。该测试只是打印出环境在 5 秒内执行的步数和循环数。该测试需要手动检查其输出：

from pettingzoo.test import performance_benchmark
from pettingzoo.butterfly import pistonball_v6
env = pistonball_v6.env()
performance_benchmark(env)

## Save Observation Test

保存观察测试是通过图形观察来目视检查游戏的观察结果，以确保它们符合预期。
我们发现观察是环境中错误的巨大来源，因此最好在可能的情况下手动检查它们。
该测试只是试图保存所有代理的观察结果。如果失败，它只会打印一条警告。需要目视检查输出的正确性。

In [ ]:
from pettingzoo.test import test_save_obs
from pettingzoo.butterfly import pistonball_v6
env = pistonball_v6.env()
test_save_obs(env)